### "Кредитный скоринг".
#### Подход Андрея. Мешок слов из триграмм

In [238]:
##### Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import hashlib
from collections import Counter

hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''

sc.stop()
conf = (SparkConf()
        .set("spark.executor.instances", 20)
        .set("spark.driver.maxResultSize", "16g")
        .set('spark.driver.memory','16g')
        .set("spark.executor.memory", '8g')
        .set("spark.yarn.executor.memoryOverhead", 1048)        
       )
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

for q in hive_config_query.split(';'):
    try:
        hc.sql(q)
    except:
        pass

In [154]:
import re

def transliterate(string):
    tr = {u'а': u'a',
           u'б': u'b',
           u'в': u'v',
           u'г': u'g',
           u'д': u'd',
           u'е': u'e',
           u'ё': u'e',
           u'ж': u'zh',
           u'з': u'z',
           u'и': u'i',
           u'й': u'y',
           u'к': u'k',
           u'л': u'l',
           u'м': u'm',
           u'н': u'n',
           u'о': u'o',
           u'п': u'p',
           u'р': u'r',
           u'с': u's',
           u'т': u't',
           u'у': u'u',
           u'ф': u'f',
           u'х': u'h',
           u'ц': u'ts',
           u'ч': u'ch',
           u'ш': u'sh',
           u'щ': u'sch',
           u'ъ': u'',
           u'ы': u'y',
           u'ь': u'',
           u'э': u'e',
           u'ю': u'yu',
           u'я': u'ya',}

    for k in tr:
        string = string.replace(k, tr[k])
        
    return string

def tri(s, cnt,d):
    s = transliterate(s.lower())
    prefix = 'D'
    for i in range(len(s)-2):
        if s[i] == '#':
            prefix = 'U'
            continue
        k = prefix + s[i:i+3]
        #if not k.isalnum():
        #if not k.isalpha():
        #    continue
        d[k] = d[k] + cnt if k in d else cnt
    return d


abc = u"abcdefghijklmnopqrstuvwxyz0123456789 !#%&'*+,-.:=_|"

def ngram_index(ngr,abc):
    n = len(abc) + 1
    return sum((abc.index(ngr[i]) if ngr[i] in abc else n) * ((n+1) ** (i-1)) for i in range(1,len(ngr))) + ((n+1) ** (len(ngr)-1)) * (ngr[0] == 'U')

In [169]:
#user_kposminin.ccall_scoring_train 
#user_kposminin.ccall_scoring_train 

from collections import Counter

table_create_query = '''
create table ccall_scoring_train as
select 
  phone_mobile, 
  call_ymd,
  max(approve) as approve,
  (unix_timestamp(max(ymd), 'yyyy-MM-dd') - unix_timestamp(min(ymd), 'yyyy-MM-dd'))/60/60/24 as ymd_range,
  stddev(unix_timestamp(ymd, 'yyyy-MM-dd')/60/60 + avg_hour) as time_std,
  count(distinct ymd) as ymd_cnt,
  count(distinct id) as id_cnt,
  avg(avg_hour) as avg_hour,
  percentile_approx(avg_hour,0.1) as avg_hour_q10,
  percentile_approx(avg_hour,0.9) as avg_hour_q90,
  urlfr,
  count(*) as cnt,
  sum(cnt) as hits,
  avg(duration) as avg_duration

from 
  user_kposminin.ccall_visits v
where
  call_ymd > ymd and call_ymd < date_add(ymd,180)
group by
  phone_mobile, 
  call_ymd,
  urlfr
'''

select_train_query = '''
select phone_mobile, call_ymd, approve, urlfr, cnt
from user_kposminin.ccall_scoring_train
'''

! hadoop fs -rm -r scoring_train_andrey
! hadoop fs -rm -r scoring_test_andrey

#Vowpal wabbit format

df_train = (hc.sql(select_train_query)
            .rdd
            .filter(lambda row: (row['call_ymd'] < '2016-12-01'))
            .map(lambda row: (row[:3],Counter(tri(row['urlfr'],row['cnt'],{}))))
            .reduceByKey(lambda v1,v2: v1 + v2)
            .map(lambda (k,v): (k,sorted([(ngram_index(vk,abc),vv) for vk,vv in v.items()])))
            .map(lambda (k,v): ('1' if k[2] == 1 else '-1') + ' | ' + ' '.join('{}:{}'.format(*e) for e in v))
            .saveAsTextFile('scoring_train_andrey')
            )

df_test = (hc.sql(select_train_query)
            .rdd
            .filter(lambda row: (row['call_ymd'] >= '2016-12-01'))
            .map(lambda row: (row[:3],Counter(tri(row['urlfr'],row['cnt'],{}))))
            .reduceByKey(lambda v1,v2: v1 + v2)
            .map(lambda (k,v): (k,sorted([(ngram_index(vk,abc),vv) for vk,vv in v.items()])))
            .map(lambda (k,v): ('1' if k[2] == 1 else '-1') + ' | ' + ' '.join('{}:{}'.format(*e) for e in v))
            .saveAsTextFile('scoring_test_andrey')
            )

rm: `scoring_train_andrey': No such file or directory
rm: `scoring_test_andrey': No such file or directory


###Дальше в LightGBM
Лучший результат: [LightGBM] [Info] Iteration: 5, training AUC : 0.608894; test : AUC : 0.628938

Данные:
* обучающая выборка ds-clickhouse01t:/data1/share/kosm/data/scoring_train_andrey_libsvm.txt
* тестирующая выборка ds-clickhouse01t:/data1/share/kosm/data/scoring_test_andrey_libsvm.txt
* конфиг файл LightGBM ds-clickhouse01t:/home/k.osminin/scoring_andrey_lgbm_files/train.conf (брался из расчета Андрея без изменений)
* Результат расчета ds-clickhouse01t:/home/k.osminin/scoring_andrey_lgbm_files/log.txt